In [ ]:
# plot_ua_plotly
# michael.smith2@nrcan-rncan.gc.ca
# Dec 2025
# Plotly emulation of metpy skewt plot for upper air data
# Written with help from CoPilot. 

# To do:
# Various stylistic and functional improvements including:
# - attempt to make the hodograph look better
# - improve wind barb appearance
# - Plot titles and station metadata for observed soundings
# - Titles and model info for model soundings
# - Add ability to overplot multiple soundings
# - Add ability to plot forecast soundings at different forecast hours
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from metpy.units import units
import metpy.calc as mpcalc

In [ ]:
# Skew-T helper functions

def skew_transform_temperature(T_c, p_hPa, skew=30.0):
    """
    Apply a simple skew transformation to temperature for Skew-T.
    This doesn't need to be physically exact; it's a visual transform:
        x = T + a * ln(p)
    where 'a' controls the skewness.
    """
    # skew is in degrees; convert to a factor that "leans" isotherms.
    # T in degC, p in hPa
    a = -np.tan(np.deg2rad(skew)) * 8  # tweak factor if you want
    T_c = np.asarray(T_c)
    p_hPa = np.asarray(p_hPa)
    return T_c + a * np.log(p_hPa / 1000.0)


def build_skewt_background(p_min=100, p_max=1000, skew=45.0):
    """
    Precompute background lines (isotherms, dry/moist adiabats, mixing ratio lines).
    Returns a dict of arrays you can use to add traces to Plotly.
    """
    p = np.linspace(p_min, p_max, 50) * units.hPa

    # Isotherms (straight lines in T-p before transform)
    T_iso = np.arange(-80, 45, 10) * units.degC

    iso_lines = []
    for T in T_iso:
        x = skew_transform_temperature(T.m, p.m, skew=skew)
        iso_lines.append((x, p.m, T.m))

    # Dry adiabats
    theta_vals = np.arange(250, 450, 10) * units.kelvin
    dry_lines = []
    for theta in theta_vals:
        T = mpcalc.dry_lapse(p, theta)
        T_c = T.to('degC').m
        x = skew_transform_temperature(T_c, p.m, skew=skew)
        dry_lines.append((x, p.m))

    # Moist adiabats
    # Start from a range of surface temperatures and integrate upward
    moist_lines = []
    T0_vals = np.arange(-10, 40, 5) * units.degC
    for T0 in T0_vals:
        T_moist = mpcalc.moist_lapse(p, T0)
        T_c = T_moist.to('degC').m
        x = skew_transform_temperature(T_c, p.m, skew=skew)
        moist_lines.append((x, p.m))

    # Mixing ratio lines (g/kg)
    w_vals = np.array([0.4, 1, 2, 4, 8, 16]) * units('g/kg')
    mix_lines = []
    for w in w_vals:
        T_mix = mpcalc.dewpoint_from_relative_humidity(
            mpcalc.temperature_from_potential_temperature(p, 300 * units.kelvin),
            0.5
        )  # dummy T, we only need shape of p
        # Use mixing ratio formula:
        e = (w / (w + 621.97 * units('g/kg'))) * p
        T_dp = mpcalc.dewpoint(e)
        T_c = T_dp.to('degC').m
        x = skew_transform_temperature(T_c, p.m, skew=skew)
        mix_lines.append((x, p.m))

    return {
        "p": p.m,
        "iso_lines": iso_lines,
        "dry_lines": dry_lines,
        "moist_lines": moist_lines,
        "mix_lines": mix_lines,
    }

In [ ]:
def plot_skewt_plotly(
    df,
    skew_type='obs',
    zoom=False,
    cutoff_elev_m=0,
    show_hodograph=False,
    skew_angle=45
):
    """
    Plot a Skew-T log-P diagram using Plotly, approximating the behavior of the
    MetPy SkewT class from your original Matplotlib function.
    """

    # Copy df to avoid modifying original
    df = df.copy()

    # Remove any data below the cutoff elevation
    if cutoff_elev_m > 0.0:
        mask = df['geopotential height_dm'].astype(float) > float(cutoff_elev_m)
        df = df[mask].reset_index(drop=True)

    # Keep these static since plotly allows zooming and panning interactively
    ylim_bottom = 1000
    ylim_top = 100

    # Populate required series
    pres = df['pressure_hPa'].values * units.hPa
    temp = df['temperature_C'].values * units.degC
    dewpoint = df['dew point temperature_C'].values * units.degC

    # Wind components
    wind_speed = df['wind speed_kmh'].values * units.km / units.hour
    wind_dir = df['wind direction_degree'].values * units.degrees
    u, v = mpcalc.wind_components(wind_speed, wind_dir)

    # Create base figure (Skew-T in first column, hodograph in second)
    specs = [[{"type": "xy"}, {"type": "xy" if show_hodograph else "domain"}]]
    col_widths = [0.7, 0.3] if show_hodograph else [1.0, 0.0]

    fig = make_subplots(
        rows=1,
        cols=2,
        specs=specs,
        column_widths=col_widths,
        horizontal_spacing=0.08,
        subplot_titles=("Skew-T Log-P", "Hodograph" if show_hodograph else "")
    )

    # Build background (you could cache this externally)
    background = build_skewt_background(p_min=ylim_top, p_max=ylim_bottom, skew=skew_angle)

    # --- Background: isotherms ---
    for x_iso, p_iso, T_label in background['iso_lines']:
        fig.add_trace(
            go.Scatter(
                x=x_iso,
                y=p_iso,
                mode="lines",
                line=dict(color="lightgray", width=1),
                hoverinfo="skip",
                showlegend=False
            ),
            row=1, col=1
        )
        # Add temp label at bottom
        if ylim_bottom >= 900:
            y_label = ylim_bottom
        else:
            y_label = background['p'].max()
        fig.add_annotation(
            x=skew_transform_temperature(T_label, y_label, skew=skew_angle),
            y=y_label,
            text=f"{T_label:.0f}°C",
            showarrow=False,
            font=dict(size=9, color="gray"),
            xref="x1", yref="y1"
        )

    # --- Background: dry adiabats ---
    for x_dry, p_dry in background['dry_lines']:
        fig.add_trace(
            go.Scatter(
                x=x_dry,
                y=p_dry,
                mode="lines",
                line=dict(color="orangered", width=1, dash="dot"),
                opacity=0.25,
                hoverinfo="skip",
                showlegend=False
            ),
            row=1, col=1
        )

    # --- Background: moist adiabats ---
    for x_moist, p_moist in background['moist_lines']:
        fig.add_trace(
            go.Scatter(
                x=x_moist,
                y=p_moist,
                mode="lines",
                line=dict(color="green", width=1, dash="dot"),
                opacity=0.25,
                hoverinfo="skip",
                showlegend=False
            ),
            row=1, col=1
        )

    # --- Background: mixing ratio lines ---
    for x_mix, p_mix in background['mix_lines']:
        fig.add_trace(
            go.Scatter(
                x=x_mix,
                y=p_mix,
                mode="lines",
                line=dict(color="steelblue", width=1, dash="dot"),
                opacity=0.25,
                hoverinfo="skip",
                showlegend=False
            ),
            row=1, col=1
        )

    # Transform temperature and dewpoint for skewed x-axis
    T_skew = skew_transform_temperature(temp.m, pres.m, skew=skew_angle)
    Td_skew = skew_transform_temperature(dewpoint.m, pres.m, skew=skew_angle)

    # --- Observed temperature ---
    fig.add_trace(
        go.Scatter(
            x=T_skew,
            y=pres.m,
            mode="lines",
            line=dict(color="red", width=2),
            name="Temperature",
            hovertemplate="P: %{y:.1f} hPa<br>T: %{customdata:.1f} °C",
            customdata=temp.m,
        ),
        row=1, col=1
    )

    # --- Dewpoint ---
    fig.add_trace(
        go.Scatter(
            x=Td_skew,
            y=pres.m,
            mode="lines",
            line=dict(color="green", width=2),
            name="Dew Point",
            hovertemplate="P: %{y:.1f} hPa<br>Td: %{customdata:.1f} °C",
            customdata=dewpoint.m,
        ),
        row=1, col=1
    )

    # --- Wind barbs (approximate with arrows / markers) ---
    # Sample at a pressure interval like your barb_interval
    barb_interval = np.arange(150, 1000, 50) * units.hPa
    ix = mpcalc.resample_nn_1d(pres, barb_interval)
    barb_pres = pres[ix].m
    barb_u = u[ix].to('knots').m
    barb_v = v[ix].to('knots').m
    barb_speed = np.sqrt(barb_u**2 + barb_v**2)

    # Place barbs at right side of plot roughly at a fixed x
    x_barb = np.full_like(barb_pres, skew_transform_temperature(30.0, barb_pres, skew=skew_angle))
    fig.add_trace(
        go.Scatter(
            x=x_barb,
            y=barb_pres,
            mode="markers",
            marker=dict(
                symbol="arrow",
                angle=np.degrees(np.arctan2(barb_u, barb_v)),  # rotate arrow into wind direction
                size=14,
                color=barb_speed,
                colorscale="Viridis",
                line=dict(width=1)
            ),
            name="Wind (kt)",
            customdata=np.vstack((barb_u, barb_v)).T,
            hovertemplate="P: %{y:.1f} hPa<br>U: %{customdata[0]:.1f} kt<br>V: %{customdata[1]:.1f} kt"
        ),
        row=1, col=1
)

    # --- LCL and parcel profile ---
    lcl_pressure, lcl_temperature = mpcalc.lcl(pres[0], temp[0], dewpoint[0])
    lcl_Tc = lcl_temperature.to('degC').m
    lcl_p = lcl_pressure.m
    lcl_x = skew_transform_temperature(lcl_Tc, lcl_p, skew=skew_angle)

    fig.add_trace(
        go.Scatter(
            x=[lcl_x],
            y=[lcl_p],
            mode="markers",
            marker=dict(color="black", size=8),
            name="LCL",
            hovertemplate="LCL<br>P: %{y:.1f} hPa<br>T: %{customdata:.1f} °C",
            customdata=[lcl_Tc],
        ),
        row=1, col=1
    )

    profile = mpcalc.parcel_profile(pres, temp[0], dewpoint[0]).to('degC')
    prof_Tc = profile.m
    prof_x = skew_transform_temperature(prof_Tc, pres.m, skew=skew_angle)

    fig.add_trace(
        go.Scatter(
            x=prof_x,
            y=pres.m,
            mode="lines",
            line=dict(color="black", width=2, dash="dash"),
            name="Parcel Profile",
            hovertemplate="P: %{y:.1f} hPa<br>Tparcel: %{customdata:.1f} °C",
            customdata=prof_Tc,
        ),
        row=1, col=1
    )

    # --- Geopotential height labels ---
    target_pressures = [1000, 900, 850, 800, 750, 700, 500, 300, 200, 100]
    pres_df = df['pressure_hPa'].dropna().values
    height_df = df['geopotential height_dm'].dropna().values  # dm or m?
    if len(pres_df) > 1 and len(height_df) > 1:
        for p in target_pressures:
            if p >= pres_df.min() and p <= pres_df.max():
                h = np.interp(p, pres_df[::-1], height_df[::-1])
                h_dm = h / 10.0  # adjust if your units differ
                fig.add_annotation(
                    x=skew_transform_temperature(-50.0, p, skew=skew_angle),
                    y=p,
                    text=f"{h_dm:.0f} dm",
                    showarrow=False,
                    font=dict(size=9, color="gray"),
                    xref="x1",
                    yref="y1"
                )

    # --- Hodograph ---
    if show_hodograph:
        # Limit to < 12 km like your original
        mask_12km = height_df < 12000
        uu = u[mask_12km].to('m/s').m
        vv = v[mask_12km].to('m/s').m
        height_12km = height_df[mask_12km]

        fig.add_trace(
            go.Scatter(
                x=uu,
                y=vv,
                mode="lines+markers",
                line=dict(color="darkslateblue", width=2),
                marker=dict(
                    size=6,
                    color=height_12km,
                    colorscale="Viridis",
                    colorbar=dict(title="Height (m)"),
                ),
                name="0–12 km wind",
                hovertemplate="U: %{x:.1f} m/s<br>V: %{y:.1f} m/s<br>Height: %{marker.color:.0f} m",
            ),
            row=1, col=2
        )

        # Clean hodograph axes
        fig.update_xaxes(
            title_text="U (m/s)",
            zeroline=True,
            zerolinewidth=1,
            mirror=True,
            showgrid=True,
            gridcolor="lightgray",
            row=1, col=2
        )
        fig.update_yaxes(
            title_text="V (m/s)",
            zeroline=True,
            zerolinewidth=1,
            mirror=True,
            showgrid=True,
            gridcolor="lightgray",
            scaleanchor="x2",  # make circular
            scaleratio=1,
            row=1, col=2
        )

    # --- Axes and layout for Skew-T ---
    fig.update_yaxes(
        title_text="Pressure (hPa)",
        type="log",
        autorange=False,
        range=[3, 2],               # 10^3 → 10^2 (i.e., 1000 → 100 hPa), top-to-bottom reversal
        tickmode="array",
        tickvals=[1000, 850, 700, 500, 300, 200, 100],
        ticktext=["1000", "850", "700", "500", "300", "200", "100"],
        showgrid=True,
        gridcolor="lightgray",
        row=1, col=1
    )




    x_min = skew_transform_temperature(-60, pres.m.max(), skew=skew_angle)
    x_max = skew_transform_temperature(40, pres.m.min(), skew=skew_angle)
    fig.update_xaxes(
        title_text="Temperature (°C, skewed)",
        range=[x_min, x_max],
        showgrid=False,
        row=1, col=1
)

    fig.update_layout(
        template="plotly_white",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="left",
            x=0.0
        ),
        height=800,
        width=1000,
        margin=dict(l=60, r=60, t=80, b=60),
    )

    return fig

In [ ]:
# Read in and transform the data for testing

df_raw = pd.read_csv('plotly_test_data.csv', sep=',', header=0)

df_raw = df_raw.rename(columns={'geopotential height_m':'geopotential height_dm'})

# Convert non-numeric data to NaN in key columns
key_cols = ['pressure_hPa', 'temperature_C', 'dew point temperature_C', 'wind speed_m/s', 'wind direction_degree']
for col in key_cols:
    if col in df_raw.columns:
        df_raw[col] = pd.to_numeric(df_raw[col], errors='coerce')

df_raw['wind speed_kmh'] = df_raw['wind speed_m/s'].values.astype(float) * 3.6 * units.km / units.h

# Remove rows with NaN in key columns
df_raw = df_raw.dropna(subset=['pressure_hPa', 'temperature_C', 'dew point temperature_C', 'relative humidity_%'])

# Reset index
df_all = df_raw.reset_index(drop=True)

In [ ]:
# Example: df must contain : 
# 'pressure_hPa', 'temperature_C', 'dew point temperature_C',
# 'wind speed_kmh', 'wind direction_degree', 'geopotential height_dm'

fig = plot_skewt_plotly(df_raw, skew_type='obs', cutoff_elev_m=0, show_hodograph=False)
fig.show()


# To save as PNG (requires kaleido installed: pip install -U kaleido)
# fig.write_image("skewt_soundings_plotly.png", width=1000, height=800, scale=2)